#### Librerias

In [1]:
from pathlib import Path

import ast
import numpy as np
import pandas as pd

#### Opciones

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)

#### Variables

In [3]:
DATA_PATH: Path = Path("../data")

## Datos

### Cargar datos

In [4]:
%%time
# Se cargan los datos `.jsonlines`
df: pd.DataFrame = pd.read_json(path_or_buf=DATA_PATH / "MLA_100k_checked_v3.jsonlines", lines=True)
df = df.replace(to_replace=["", None], value=pd.NA)
print(f"Filas: {df.shape[0]:,} | Columnas: {df.shape[1]:,}")
df.head(5)

Filas: 100,000 | Columnas: 45
CPU times: user 5.19 s, sys: 569 ms, total: 5.76 s
Wall time: 5.81 s


,seller_address,warranty,sub_status,condition,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,variations,site_id,listing_type_id,price,attributes,buying_mode,tags,listing_source,parent_item_id,coverage_areas,category_id,descriptions,last_updated,international_delivery_mode,pictures,id,official_store_id,differential_pricing,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",<NA>,[],new,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8208882349,[],MLA,bronze,80.0,[],buy_it_now,[dragged_bids_and_visits],<NA>,MLA6553902747,[],MLA126406,[{'id': 'MLA4695330653-912855983'}],2015-09-05T20:42:58.000Z,none,"[{'size': '500x375', 'secure_url': 'https://a2...",MLA4695330653,NaN,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-04 20:42:53,active,<NA>,NaN,NaN,1,2015-09-05 20:42:53,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NUESTRA REPUTACION,[],used,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8141699488,[],MLA,silver,2650.0,[],buy_it_now,[],<NA>,MLA7727150374,[],MLA10267,[{'id': 'MLA7160447179-930764806'}],2015-09-26T18:08:34.000Z,none,"[{'size': '499x334', 'secure_url': 'https://a2...",MLA7160447179,NaN,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-25 18:08:30,active,<NA>,NaN,NaN,1,2015-09-26 18:08:30,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",<NA>,[],used,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8386096505,[],MLA,bronze,60.0,[],buy_it_now,[dragged_bids_and_visits],<NA>,MLA6561247998,[],MLA1227,[{'id': 'MLA7367189936-916478256'}],2015-09-09T23:57:10.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7367189936,NaN,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-08 23:57:07,active,<NA>,NaN,NaN,1,2015-09-09 23:57:07,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",<NA>,[],new,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",5377752182,[],MLA,silver,580.0,[],buy_it_now,[],<NA>,<NA>,[],MLA86345,[{'id': 'MLA9191625553-932309698'}],2015-10-05T16:03:50.306Z,none,"[{'size': '441x423', 'secure_url': 'https://a2...",MLA9191625553,NaN,NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,2015-12-04 01:13:16,active,<NA>,NaN,NaN,1,2015-09-28 18:47:56,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",MI REPUTACION.,[],used,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",2938071313,[],MLA,bronze,30.0,[],buy_it_now,[dragged_bids_and_visits],<NA>,MLA3133256685,[],MLA41287,[{'id': 'MLA7787961817-902981678'}],2015-08-28T13:37:41.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7787961817,NaN,NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,Serenata - Jennifer Blake,False,2015-08-24T2

In [5]:
# Uso de memoria
print(f"Uso de memoria: {df.memory_usage().sum() / 1_000_000:.2f} MB")

Uso de memoria: 34.60 MB


### Tipos de datos y Valores faltantes

In [6]:
pd.concat(
    objs=[
        df.dtypes.rename("TiposDeDatos"),
        df.isna().sum().mul(100 / 100_000).rename("ValoresFaltantes%"),
        df.astype("str").nunique().rename("ValoresUnicos"),
    ],
    axis=1,
).sort_values("ValoresFaltantes%", ascending=False)

,TiposDeDatos,ValoresFaltantes%,ValoresUnicos
listing_source,object,100.000,1
differential_pricing,float64,100.000,1
subtitle,float64,100.000,1
catalog_product_id,float64,99.989,10
original_price,float64,99.857,115
official_store_id,float64,99.182,199
video_id,object,97.015,2078
warranty,object,60.897,10264
parent_item_id,object,23.011,76990
secure_thumbnail,object,0.789,99172


In [7]:
# Se quiere asignar el tipo de dato correcto
FLOATS: list = ["catalog_product_id", "original_price", "official_store_id", "base_price", "price"]
INTS: list = ["seller_id", "initial_quantity", "sold_quantity", "available_quantity"]
BOOLS: list = ["accepts_mercadopago", "automatic_relist"]
CATEGORICALS: list = ["condition", "listing_type_id", "buying_mode", "currency_id", "status"]
DATES: list = ["last_updated", "date_created", "stop_time", "start_time"]
STRINGS: list = [
    "video_id",
    "warranty",
    "parent_item_id",
    "category_id",
    "thumbnail",
    "secure_thumbnail",
    "title",
    "permalink",
    "id",
]
OTHERS: list = [
    "seller_address",
    "sub_status",
    "deal_ids",
    "attributes",
    "variations",
    "non_mercado_pago_payment_methods",
    "shipping",
    "descriptions",
    "tags",
    "pictures",
]

In [8]:
# Se eliminan variables completamente vacias.
EMPTY: list = ["differential_pricing", "subtitle", "international_delivery_mode", "coverage_areas", "listing_source"]
df = df.drop(columns=EMPTY)

# Se eliminan variables que no varian.
NON_VARIANT: list = ["site_id"]
df = df.drop(columns=NON_VARIANT)

In [9]:
# Se asignan los tipos de datos correctos
df[FLOATS] = df[FLOATS].astype(dtype=float)
df[INTS] = df[INTS].astype(dtype=int)
df[BOOLS] = df[BOOLS].astype(dtype=bool)
df[CATEGORICALS] = df[CATEGORICALS].astype(dtype="category")
df[STRINGS] = df[STRINGS].astype(dtype="object")
for var in DATES:
    df[var] = pd.to_datetime(df[var])

In [10]:
# Uso de memoria
print(f"Uso de memoria: {df.memory_usage().sum() / 1_000_000:.2f} MB")

Uso de memoria: 26.30 MB


### Correccion de caracteristicas

#### `seller_address`

In [11]:
# Se extrae informacion de Pais, Ciudad y Estado
address_df: pd.DataFrame = (
    pd.json_normalize(data=df["seller_address"])
    .replace(to_replace="", value=pd.NA)
    .drop(columns=["country.id", "state.id", "city.id"])
)
print("Pais:")
print(address_df["country.name"].value_counts(dropna=False).to_dict(), end="\n\n")
print("Estado:")
print(address_df["state.name"].value_counts(dropna=False).to_dict(), end="\n\n")
print("Ciudad:")
print(address_df["city.name"].value_counts(dropna=False).nsmallest(50).to_dict())

Pais:
{'Argentina': 99997, <NA>: 3}

Estado:
{'Capital Federal': 57854, 'Buenos Aires': 35013, 'Santa Fe': 2672, 'Córdoba': 1908, 'Mendoza': 450, 'Chubut': 375, 'Entre Ríos': 287, 'Tucumán': 237, 'San Juan': 151, 'Salta': 146, 'Misiones': 143, 'Río Negro': 140, 'Corrientes': 124, 'Neuquén': 101, 'La Pampa': 86, 'Chaco': 77, 'San Luis': 60, 'Jujuy': 40, 'Santiago del Estero': 29, 'Formosa': 28, 'Santa Cruz': 25, 'Catamarca': 21, 'La Rioja': 18, 'Tierra del Fuego': 12, <NA>: 3}

Ciudad:
{'FEDRACION': 1, 'Vicente López, palermo, Belgrano, Colegiales': 1, 'CABA (Belgrano)': 1, 'VILLA SARMIENTO': 1, 'Bahia  Blanca': 1, 'ALMAGRO de 13 hs a 19 hs de lunes a viernes': 1, 'FlorencioVarelayVillaDominico': 1, 'Caballito - Villa del Parque': 1, 'Almagro (una vez por semana) y Quilmes Oeste (resto de los dias) A COORDINAR': 1, 'CUIDADELA': 1, 'Capitan Bermudez': 1, 'picotruncado': 1, 'Granja de Funez': 1, 'evita': 1, 'COGHLAN - CAPITAL FEDERAL': 1, 'el bolson': 1, 'Captal Federal': 1, 'General San 

- Se utilizara unicamente `Estado`, ya que `Pais` es 99% Argentina (y posiblemente los NA tambien pertenezcan al mismo pais) y la calidad del dato de `Ciudad` no es la mejor. Dado el corto tiempo de la prueba, solucionar el problema de la calidad queda por fuera del alcance.

In [12]:
df = pd.concat(objs=[df, address_df[["state.name"]]], axis=1)
df = df.drop(columns="seller_address")
df["state.name"] = df["state.name"].astype(dtype="category")

#### `sub_status`

In [13]:
df["sub_status"] = df["sub_status"].str[0].replace(to_replace=np.nan, value=pd.NA).astype(dtype="category")
print(df["sub_status"].value_counts(dropna=False).to_dict())

{nan: 99014, 'suspended': 966, 'expired': 13, 'deleted': 7}


#### `deal_ids`

In [14]:
df["deal_ids"] = df["deal_ids"].str[0].replace(to_replace=np.nan, value=pd.NA)
print(df["deal_ids"].value_counts(dropna=False).to_dict())

{<NA>: 99760, 'MOSH6': 60, 'WGPLA': 41, '15H9O': 28, 'ABQ1I': 28, 'LZFND': 22, 'JXLIQ': 7, 'MISCR': 6, 'CHMGI': 6, 'R99GD': 5, 'GAC6A': 5, 'GRMWV': 3, 'AZEBL': 3, 'UQU1U': 3, 'ZZMKC': 2, 'BBLRY': 2, 'UKXWI': 2, 'UUINF': 2, 'VYYJI': 2, 'PPW2S': 1, 'YOLH8': 1, '7FOQD': 1, 'WO4YE': 1, 'Y3MPW': 1, '805OS': 1, 'ZZHBN': 1, 'QXDHA': 1, 'SI7I0': 1, 'W40QU': 1, 'F9G6U': 1, 'CULNX': 1, 'VR2RD': 1}


#### `pictures`

In [15]:
# Se construye la variable numero de imagenes
df["n_pictures"] = np.vectorize(len)(df["pictures"])
df = df.drop(columns="pictures")

- Como no se tiene acceso a los links de las imagenes, se construye la variable `n_pictures` para recuperar parte de la informacion de la variable.

#### `tags`

In [16]:
tags_df = pd.get_dummies(df["tags"].explode()).groupby(level=0).sum()
tags_df.head()

,dragged_bids_and_visits,dragged_visits,free_relist,good_quality_thumbnail,poor_quality_thumbnail
0,1,0,0,0,0
1,0,0,0,0,0
2,1,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0


In [17]:
df = pd.concat(objs=[df, tags_df], axis=1)
df = df.drop(columns="tags")

- Se crean variables dummy para cada tag. Como no son tantos, no corremos peligro de maldicion de dimensionalidad.

#### `descriptions`

In [18]:
# Solo tienen 1 diccionario en la lista o no tienen
df[df["descriptions"].str.len() != 1]["descriptions"].astype("str").unique()

array(['[]'], dtype=object)

In [19]:
# Los diccionarios solo tienen 1 key -> `id`
_ = [ast.literal_eval(e) for e in df["descriptions"].str[0].values.tolist() if str(e) != "nan"]
{list(dictionario.keys())[0] for dictionario in _}

{'id'}

In [20]:
df["descriptions"] = df["descriptions"].str[0].map(lambda x: ast.literal_eval(x)["id"] if str(x) != "nan" else pd.NA)
df[["id", "descriptions"]].head()

,id,descriptions
0,MLA4695330653,MLA4695330653-912855983
1,MLA7160447179,MLA7160447179-930764806
2,MLA7367189936,MLA7367189936-916478256
3,MLA9191625553,MLA9191625553-932309698
4,MLA7787961817,MLA7787961817-902981678


- `descriptions` parece ser el mismo `id` seguido de otro numero identificador

#### `shipping`

In [21]:
shipping_df = pd.json_normalize(df["shipping"])
shipping_df.head()

,local_pick_up,methods,tags,free_shipping,mode,dimensions,free_methods
0,True,[],[],False,not_specified,None,NaN
1,True,[],[],False,me2,None,NaN
2,True,[],[],False,me2,None,NaN
3,True,[],[],False,me2,None,NaN
4,True,[],[],False,not_specified,None,NaN


In [22]:
shipping_df["local_pick_up"].value_counts(dropna=False)

local_pick_up
True     79561
False    20439
Name: count, dtype: int64

In [23]:
shipping_df["methods"].value_counts(dropna=False)

methods
[]      96992
NaN      3006
None        2
Name: count, dtype: int64

In [24]:
shipping_df["tags"].value_counts(dropna=False)

tags
[]                               99931
[relisted_with_modifications]       59
[free_method_upgraded]              10
Name: count, dtype: int64

In [25]:
shipping_df["free_shipping"].value_counts(dropna=False)

free_shipping
False    96984
True      3016
Name: count, dtype: int64

In [26]:
shipping_df["mode"].value_counts(dropna=False)

mode
me2              51255
not_specified    45171
custom            3481
me1                 93
Name: count, dtype: int64

In [27]:
shipping_df["dimensions"].value_counts(dropna=False)

dimensions
None             99977
33x20x12,300         4
33x20x12,100         3
30x30x30,650         3
33x20x12,400         2
33x20x12,80          2
33x20x12,1000        2
5x25x25,150          1
33x20x12,500         1
10x10x20,700         1
10x20x20,350         1
5x20x30,650          1
33x20x12,800         1
15x30x45,650         1
Name: count, dtype: int64

In [28]:
shipping_df["free_methods"].value_counts(dropna=False)

free_methods
NaN                                                                                                                                 96994
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73328}]                                                                     2955
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73330}]                                                                       40
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 501145}]                                                                       6
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 501146}]                                                                       4
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73328}, {'rule': {'value': None, 'free_mode': 'country'}, 'id': 73330}]        1
Name: count, dtype: int64

In [29]:
shipping_df = shipping_df.drop(columns=["methods", "dimensions", "free_methods"]).rename(columns=lambda x: f"shipping__{x}")
shipping_df["shipping__tags"] = shipping_df["shipping__tags"].str[0]
df = pd.concat(objs=[df, shipping_df], axis=1)
df = df.drop(columns="shipping")

- Se rescatan la informacion de `local_pick_up`, `tags`, `free_shipping` y `mode`
- No se encuentra una forma clara de rescatar la informacion de `free_methods`

#### `non_mercado_pago_payment_methods`

In [54]:
nmppm_df = pd.get_dummies(df["non_mercado_pago_payment_methods"].map(lambda x: [element["description"] for element in x]).explode()).groupby(level=0).sum().rename(columns=lambda x: f"non_mercado_pago_payment_methods__{x.lower().replace(' ', '_')}")
nmppm_df.head()

,non_mercado_pago_payment_methods__acordar_con_el_comprador,non_mercado_pago_payment_methods__american_express,non_mercado_pago_payment_methods__cheque_certificado,non_mercado_pago_payment_methods__contra_reembolso,non_mercado_pago_payment_methods__diners,non_mercado_pago_payment_methods__efectivo,non_mercado_pago_payment_methods__giro_postal,non_mercado_pago_payment_methods__mastercard,non_mercado_pago_payment_methods__mastercard_maestro,non_mercado_pago_payment_methods__mercadopago,non_mercado_pago_payment_methods__tarjeta_de_crédito,non_mercado_pago_payment_methods__transferencia_bancaria,non_mercado_pago_payment_methods__visa,non_mercado_pago_payment_methods__visa_electron
0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0,1,1,0,0
3,0,0,0,0,0,1,0,0,0,0,1,1,0,0
4,1,0,0,0,0,1,0,0,0,0,0,1,0,0


In [31]:
df = pd.concat(objs=[df, nmppm_df], axis=1)
df = df.drop(columns="non_mercado_pago_payment_methods")

#### `variations` y `attributes`

In [49]:
(df["variations"].str.len() > 0).astype(int).value_counts(True)

variations
0    0.91808
1    0.08192
Name: proportion, dtype: float64

In [50]:
(df["attributes"].str.len() > 0).astype(int).value_counts(True)

attributes
0    0.87575
1    0.12425
Name: proportion, dtype: float64

In [55]:
df.loc[99997, "variations"]

[{'attribute_combinations': [{'value_id': '92025',
    'name': 'Color Primario',
    'value_name': 'Negro',
    'id': '83000'},
   {'value_id': '101995', 'name': 'Talle', 'value_name': 'M', 'id': '93000'}],
  'seller_custom_field': None,
  'picture_ids': ['990801-MLA20408435180_092015'],
  'sold_quantity': 0,
  'available_quantity': 1,
  'id': 9473023844,
  'price': 600}]

In [56]:
df.loc[99997, "attributes"]

[{'value_id': 'male',
  'attribute_group_id': 'DFLT',
  'name': 'Género',
  'value_name': 'Hombre',
  'attribute_group_name': 'Otros',
  'id': 'GENDER'},
 {'value_id': 'Season-Autumn-Winter',
  'attribute_group_id': 'FIND',
  'name': 'Season',
  'value_name': 'Autumn-Winter',
  'attribute_group_name': 'Ficha técnica',
  'id': 'Season'}]

- Se decide eliminar estas dos variables ya que no es claro como se pueden disponibilizar para el entrenamiento.

In [57]:
df = df.drop(columns=["variations", "attributes"])

In [59]:
# Uso de memoria
print(f"Uso de memoria: {df.memory_usage().sum() / 1_000_000:.2f} MB")

# Shape final
df.shape

Uso de memoria: 37.90 MB


(100000, 57)

### Duplicados

In [62]:
df.duplicated().sum()

np.int64(0)

- No hay duplicados

## Guardar datos en formato mas eficiente.

In [63]:
df.to_parquet(path=DATA_PATH / "data.parquet.gzip", engine="fastparquet", compression="gzip")

In [64]:
%%time
# Se cargan los datos `.jsonlines`
df: pd.DataFrame = pd.read_parquet(path=DATA_PATH / "data.parquet.gzip", engine="fastparquet")
print(f"Filas: {df.shape[0]:,} | Columnas: {df.shape[1]:,}")
df.head(5)

Filas: 100,000 | Columnas: 57
CPU times: user 182 ms, sys: 10.1 ms, total: 193 ms
Wall time: 192 ms


,warranty,sub_status,condition,deal_ids,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,category_id,descriptions,last_updated,id,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,initial_quantity,start_time,permalink,sold_quantity,available_quantity,state.name,n_pictures,dragged_bids_and_visits,dragged_visits,free_relist,good_quality_thumbnail,poor_quality_thumbnail,shipping__local_pick_up,shipping__tags,shipping__free_shipping,shipping__mode,non_mercado_pago_payment_methods__acordar_con_el_comprador,non_mercado_pago_payment_methods__american_express,non_mercado_pago_payment_methods__cheque_certificado,non_mercado_pago_payment_methods__contra_reembolso,non_mercado_pago_payment_methods__diners,non_mercado_pago_payment_methods__efectivo,non_mercado_pago_payment_methods__giro_postal,non_mercado_pago_payment_methods__mastercard,non_mercado_pago_payment_methods__mastercard_maestro,non_mercado_pago_payment_methods__mercadopago,non_mercado_pago_payment_methods__tarjeta_de_crédito,non_mercado_pago_payment_methods__transferencia_bancaria,non_mercado_pago_payment_methods__visa,non_mercado_pago_payment_methods__visa_electron
0,None,NaN,new,None,80.0,8208882349,bronze,80.0,buy_it_now,MLA6553902747,MLA126406,MLA4695330653-912855983,2015-09-05 20:42:58+00:00,MLA4695330653,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05 20:42:53+00:00,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-04 20:42:53,active,None,NaN,1,2015-09-05 20:42:53,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1,Capital Federal,2,1,0,0,0,0,True,None,False,not_specified,1,0,0,0,0,1,0,0,0,0,0,1,0,0
1,NUESTRA REPUTACION,NaN,used,None,2650.0,8141699488,silver,2650.0,buy_it_now,MLA7727150374,MLA10267,MLA7160447179-930764806,2015-09-26 18:08:34+00:00,MLA7160447179,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26 18:08:30+00:00,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-25 18:08:30,active,None,NaN,1,2015-09-26 18:08:30,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1,Capital Federal,6,0,0,0,0,0,True,None,False,me2,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,None,NaN,used,None,60.0,8386096505,bronze,60.0,buy_it_now,MLA6561247998,MLA1227,MLA7367189936-916478256,2015-09-09 23:57:10+00:00,MLA7367189936,NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09 23:57:07+00:00,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,2015-11-08 23:57:07,active,None,NaN,1,2015-09-09 23:57:07,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1,Capital Federal,1,1,0,0,0,0,True,None,False,me2,0,0,0,0,0,1,0,0,0,0,1,1,0,0
3,None,NaN,new,None,580.0,5377752182,silver,580.0,buy_it_now,None,MLA86345,MLA9191625553-932309698,2015-10-05 16:03:50.306000+00:00,MLA9191625553,NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28 18:47:56+00:00,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,2015-12-04 01:13:16,active,None,NaN,1,2015-09-28 18:47:56,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1,Capital Federal,2,0,0,0,0,0,True,None,False,me2,0,0,0,0,0,1,0,0,0,0,1,1,0,0
4,MI REPUTACION.,NaN,used,None,30.0,2938071313,bronze,30.0,buy_it_now,MLA3133256685,MLA41287,MLA7787961817-902981678,2015-08-28 13:37:41+00:00,MLA7787961817,NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,Serenata - Jennifer Blake,False,2015-08-24 22:07:20+00:00,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,2015-10-23 22:07:20,active,None,NaN,1,2015-08-24 22:07:20,http://articulo.mercadolibre.com.ar/MLA7787961...,0,1,Buenos Aires,2,1,0,0,0,0,True,None,False,not_specified,1,0,0,0,0,1,0,0,0,0,0,1,0,0


In [66]:
!ls -lh {DATA_PATH}

total 273M
-rwxrwxrwx 1 jjmov99 jjmov99 259M Jul 12 18:09 MLA_100k_checked_v3.jsonlines
-rw-r--r-- 1 jjmov99 jjmov99  15M Jul 13 23:29 data.parquet.gzip
